# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [56]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [57]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [58]:
symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f'{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 70795550, 'calculationPrice': 'close', 'change': -0.85, 'changePercent': -0.0058, 'close': 150.56, 'closeSource': 'oifficla', 'closeTime': 1678305707231, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 154.85, 'delayedPriceTime': 1636352419613, 'extendedChange': 0.04, 'extendedChangePercent': 0.00028, 'extendedPrice': 154.16, 'extendedPriceTime': 1650683656384, 'high': 154.23, 'highSource': 'aedn dy e1clte5mi priue', 'highTime': 1702123250457, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 148.691, 'iexCloseTime': 1657735307658, 'iexLastUpdated': 1711161367468, 'iexMarketPercent': 0.01120577520355498, 'iexOpen': 149.61, 'iexOpenTime': 1660116984854, 'iexRealtimePrice': 154.41, 'iexRealtimeSize': 10, 'iexVolume': 530492, 'lastTradeTime': 1707980556735, 'latestPrice': 148.9, 'latestSource': 'Close', 'latestTime': 'August 26, 2021', 'latestUpdate': 1657203956375, 'latestVolume': 49815221, 'low': 150.56, 'lowSource': 'ip 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [59]:
price = data['latestPrice']
pe_ratio = data['peRatio']
price, pe_ratio

(148.9, 29.78)

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [60]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i: i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [61]:
def quantitative_value_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f'{base_url}/stock/market/batch?symbols={symbols}&types=quote&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['quote']['latestPrice']
            pe_ratio = data[symbol]['quote']['peRatio']
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        pe_ratio,
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe.fillna(0)

final_dataframe = quantitative_value_data()
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,181.01,58.22,0.0
1,AAL,20.50,-2.04,0.0
2,AAP,213.72,23.39,0.0
3,AAPL,154.08,29.73,0.0
4,ABBV,125.00,32.84,0.0
...,...,...,...,...
500,YUM,137.40,30.65,0.0
501,ZBH,150.79,34.15,0.0
502,ZBRA,592.48,41.31,0.0
503,ZION,58.04,5.52,0.0


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [62]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,76.61,4.12,0.0
1,MPC,58.36,5.12,0.0
2,ZION,58.04,5.52,0.0
3,GM,49.88,5.80,0.0
4,PRU,106.04,5.90,0.0
5,BIO,810.74,6.24,0.0
6,AFL,56.90,6.91,0.0
7,COF,171.97,6.97,0.0
8,CINF,127.53,7.16,0.0
9,CE,162.45,7.40,0.0


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [64]:
def portfolio_input():
    global portfolio_size
    portfolio_size = 1000000 # input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [65]:
portfolio_input()
portfolio_size

1000000

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [67]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for row in final_dataframe.index:
    price = final_dataframe.loc[row, 'Price']
    buy_count =  math.floor(position_size / price)
    final_dataframe.loc[row, 'Number of Shares to Buy'] = buy_count

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,76.61,4.12,261.0
1,MPC,58.36,5.12,342.0
2,ZION,58.04,5.52,344.0
3,GM,49.88,5.80,400.0
4,PRU,106.04,5.90,188.0
5,BIO,810.74,6.24,24.0
6,AFL,56.90,6.91,351.0
7,COF,171.97,6.97,116.0
8,CINF,127.53,7.16,156.0
9,CE,162.45,7.40,123.0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: